In [25]:
import pandas as pd

In [26]:
# read the data into a pandas dataframe
import os
def data2df (path, label):
    file, text = [], []
    for f in os.listdir(path):
        file.append(f)
        fhr = open(path+f, 'r',encoding='utf-8', errors='ignore') 
        t = fhr.read()
        text.append(t)
        fhr.close()
    return(pd.DataFrame({'file': file, 'text': text, 'class':label}))

dfnonpro = data2df('C:\\Users\\LENOVO\\Documents\\Varun\\ASU\\Wolf Pack\\CIS 509\\Week 6\\Assignment 5\\HealthProNonPro\\HealthProNonPro\\NonPro\\', 0) # Nonpro
dfpro    = data2df('C:\\Users\\LENOVO\\Documents\\Varun\\ASU\\Wolf Pack\\CIS 509\\Week 6\\Assignment 5\\HealthProNonPro\\HealthProNonPro\\Pro\\', 1) # Pro

df = pd.concat([dfpro, dfnonpro], axis=0)
df.sample(frac=0.005)

,file,text,class
682,a31606.txt,"ive never tried it. But i read experimention, ...",0
1432,a69565.txt,i want want to be right with god and also i wo...,0
1150,a61450.txt,Sugar cannot be healthy. Sugar cane can be he...,0
1479,a69614.txt,huh???\neating them... did u do it???\nplzzzzz...,0
1402,a69535.txt,To have the highest point count on yahoo answe...,0
1579,a7367.txt,"great answer, lets vote",0
447,a24679.txt,It appears that the question period has expire...,0
155,a24387.txt,"Its easy. Just eat a healthy, plant-based diet...",0
1678,ans822.txt,Lamictal (lamotrigine) can cause side-effects ...,1
824,ans1740.txt,While your headaches and feeling faint may be ...,1


In [27]:
# setup the data
X, y = df['text'], df['class']
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=1) #30% randomly slected instances in test set

In [28]:
# setup a custom preprocessor
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def preprocess(text):
    # replace one or more white-space characters with a space
    regex = re.compile(r"\s+")                               
    text = regex.sub(' ', text)    
    # lower case
    text = text.lower()          
    # remove digits and punctuation
    regex = re.compile(r"[%s%s]" % (string.punctuation, string.digits))
    text = regex.sub(' ', text)           
    # remove stop words
    sw = stopwords.words('english')
    text = text.split()                                              
    text = ' '.join([w for w in text if w not in sw]) 
    # remove short words
    ' '.join([w for w in text.split() if len(w) >= 2])
    # lemmatize as stemming leads to loss of vital information at times 
    text = ' '.join([(WordNetLemmatizer()).lemmatize(w) for w in text.split()]) 
    return text

In [29]:
# setup the preprocessing->model pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
clf = Pipeline(steps=[
    ('pp', TfidfVectorizer(
        preprocessor=preprocess,
        lowercase=True, stop_words='english', 
        use_idf=True, smooth_idf=True,
        min_df=1, max_df=1.0, max_features=None, 
        ngram_range=(1, 1))),
    ('mdl',     MultinomialNB())
    ])

In [30]:
# setup grid search
from sklearn.model_selection import GridSearchCV
param_grid = {
    'pp__norm': ['l1','l2',None],
    'mdl__alpha':[0.01, 0.1, 0.2, 0.5, 1]
}
gscv = GridSearchCV(clf, param_grid, iid=False, cv=5, return_train_score=False)

In [31]:
# search for best parameters/estimator
gscv.fit(Xtrain, ytrain)

print(gscv.best_estimator_, "\n")

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated token

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated token

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated token

Pipeline(memory=None,
     steps=[('pp', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...se_idf=True, vocabulary=None)), ('mdl', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))]) 



In [32]:
# evaluate best_estimator_ on test data
ypred = gscv.best_estimator_.predict(Xtest)
from sklearn import metrics
print (metrics.accuracy_score(ytest, ypred))
print (metrics.confusion_matrix(ytest, ypred))
print (metrics.classification_report(ytest, ypred))

0.9463148316651502
[[476  54]
 [  5 564]]
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       530
           1       0.91      0.99      0.95       569

   micro avg       0.95      0.95      0.95      1099
   macro avg       0.95      0.94      0.95      1099
weighted avg       0.95      0.95      0.95      1099

